In [ ]:
!pip install gurobipy

In [3]:
import pandas as pd
from gurobipy import *


In [4]:
# Reading the data from the excel sheets
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name="PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name="PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name="Truck")

# Convert relevant columns to datetime and normalize shift times
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])



In [24]:
# Initialize the model
model = Model("DHL_Optimization")

# Decision Variables
source_list = list(trucking_df['Origin_ID'].unique()[:2])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:2])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list = [
    x for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]

valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(100)

# Decision Variables
X = {}
Z = {}
T = {}
ArrivalDay = {}
ArrivalTime = {}

for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]:
    X[(i, j, k, l)] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}_{l}")
    ArrivalTime[(i, j, k, l)] = model.addVar(lb=0, ub=24, vtype=GRB.CONTINUOUS, name=f"ArrivalTime_{i}_{j}_{k}_{l}")

for l in trucks:
    Z[l] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}")
    T[l] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"T_{l}")
    ArrivalDay[l] = model.addVar(lb = 0, vtype=GRB.INTEGER, name=f"ArrivalDay_{l}")


In [25]:
# Objective function: Minimize the total arrival days
model.setObjective(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations for l in trucks) - quicksum(ArrivalDay[l] for l in trucks), GRB.MAXIMIZE)

In [26]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations) <= 2 * Z[l])

In [27]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model.addConstr(T[l] >= release_time * X[(i, j, k, l)])

In [28]:

# 3. Truck must arrive at the destination within the operational hours
for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    for l in trucks:
        arrival_time_var = ArrivalTime[(i, j, k, l)]
        model.addConstr(arrival_time_var == (T[l] + travel_time * X[(i, j, k, l)] + 24 * ArrivalDay[l]) - 24 * model.addVar(lb = 0, vtype=GRB.INTEGER, name=f"multiplier_{i}_{j}_{k}_{l}"))
        model.addConstr(arrival_time_var >= start_shift)
        model.addConstr(arrival_time_var <= end_shift)

In [29]:
# 4. Each consignment must be assigned to exactly one truck
for k in consignment_list:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j) in routes_list for l in trucks if (i, j, k) in valid_combinations) == 1)

In [18]:
# 5. Flow conservation: If a truck leaves a source, it must go to one destination
for l in trucks:
    for i in source_list:
        model.addConstr(quicksum(X[(i, j, k, l)] for j in destination_list if j != i for k in consignment_list if (i, j, k) in valid_combinations) >= Z[l])

In [14]:
# 6. Sorting Capacity: Each PZE should have enough capacity to accommodate all the incoming trucks
for j in destination_list:
    working_hours = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0] - destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    model.addConstr(quicksum(X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0]
                      for i in source_list if j != i for k in consignment_list for l in trucks if (i, j, k) in valid_combinations ) <= working_hours * destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0])

AttributeError: Index out of range for attribute 'X'

In [30]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22155 rows, 16800 columns and 60600 nonzeros
Model fingerprint: 0x21514c37
Variable types: 5500 continuous, 11300 integer (5600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 11000 rows and 100 columns
Presolve time: 0.13s
Presolved: 11155 rows, 16700 columns, 53200 nonzeros
Variable types: 5500 continuous, 11200 integer (5500 binary)

Root relaxation: objective 5.500000e+01, 464 iterations, 0.05 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   55.0

In [1]:
# Print the objective value and solution
print("Objective value:", model.objVal)

solution = {}
for v in model.getVars():
    if v.x > 0:
        solution[v.varName] = v.x

print("Solution:", solution)

NameError: name 'model' is not defined